In [346]:
# Can have as many cells as you want for code
import pandas as pd
import numpy as np
filepath = "./data/catB_train.parquet" 
# the initialised filepath MUST be a relative path to a folder named data that contains the parquet file
df = pd.read_parquet(filepath)
df.head().style

,clntnum,race_desc,ctrycode_desc,clttype,stat_flag,min_occ_date,cltdob_fix,cltsex_fix,flg_substandard,flg_is_borderline_standard,flg_is_revised_term,flg_is_rental_flat,flg_has_health_claim,flg_has_life_claim,flg_gi_claim,flg_is_proposal,flg_with_preauthorisation,flg_is_returned_mail,is_consent_to_mail,is_consent_to_email,is_consent_to_call,is_consent_to_sms,is_valid_dm,is_valid_email,is_housewife_retiree,is_sg_pr,is_class_1_2,is_dependent_in_at_least_1_policy,f_ever_declined_la,hh_20,pop_20,hh_size,hh_size_est,annual_income_est,n_months_last_bought_products,flg_latest_being_lapse,flg_latest_being_cancel,recency_lapse,recency_cancel,tot_inforce_pols,tot_cancel_pols,ape_gi_42e115,ape_ltc_1280bf,ape_grp_6fc3e6,ape_grp_de05ae,ape_inv_dcd836,ape_grp_945b5a,ape_grp_6a5788,ape_ltc_43b9d5,ape_grp_9cdedf,ape_lh_d0adeb,ape_grp_1581d7,ape_grp_22decf,ape_lh_507c37,ape_lh_839f8a,ape_inv_e9f316,ape_gi_a10d1b,ape_gi_29d435,ape_grp_caa6ff,ape_grp_fd3bfb,ape_lh_e22a6a,ape_grp_70e1dd,ape_grp_e04c3a,ape_grp_fe5fb8,ape_gi_856320,ape_grp_94baec,ape_gi_058815,ape_grp_e91421,ape_lh_f852af,ape_lh_947b15,ape_32c74c,sumins_gi_42e115,sumins_ltc_1280bf,sumins_grp_6fc3e6,sumins_grp_de05ae,sumins_inv_dcd836,sumins_grp_945b5a,sumins_grp_6a5788,sumins_ltc_43b9d5,sumins_grp_9cdedf,sumins_lh_d0adeb,sumins_grp_1581d7,sumins_grp_22decf,sumins_lh_507c37,sumins_inv_e9f316,sumins_gi_a10d1b,sumins_gi_29d435,sumins_grp_caa6ff,sumins_grp_fd3bfb,sumins_lh_e22a6a,sumins_grp_70e1dd,sumins_grp_e04c3a,sumins_grp_fe5fb8,sumins_gi_856320,sumins_grp_94baec,sumins_gi_058815,sumins_grp_e91421,sumins_lh_f852af,sumins_lh_947b15,sumins_32c74c,prempaid_gi_42e115,prempaid_ltc_1280bf,prempaid_grp_6fc3e6,prempaid_grp_de05ae,prempaid_inv_dcd836,prempaid_grp_945b5a,prempaid_grp_6a5788,prempaid_ltc_43b9d5,prempaid_grp_9cdedf,prempaid_lh_d0adeb,prempaid_grp_1581d7,prempaid_grp_22decf,prempaid_lh_507c37,prempaid_lh_839f8a,prempaid_inv_e9f316,prempaid_gi_a10d1b,prempaid_gi_29d435,prempaid_grp_caa6ff,prempaid_grp_fd3bfb,prempaid_lh_e22a6a,prempaid_grp_70e1dd,prempaid_grp_e04c3a,prempaid_grp_fe5fb8,prempaid_gi_856320,prempaid_grp_94baec,prempaid_gi_058815,prempaid_grp_e91421,prempaid_lh_f852af,prempaid_lh_947b15,prempaid_32c74c,ape_839f8a,ape_e22a6a,ape_d0adeb,ape_c4bda5,ape_ltc,ape_507c37,ape_gi,f_hold_839f8a,f_hold_e22a6a,f_hold_d0adeb,f_hold_c4bda5,f_hold_ltc,f_hold_507c37,f_hold_gi,sumins_839f8a,sumins_e22a6a,sumins_d0adeb,sumins_c4bda5,sumins_ltc,sumins_507c37,sumins_gi,prempaid_839f8a,prempaid_e22a6a,prempaid_d0adeb,prempaid_c4bda5,prempaid_ltc,prempaid_507c37,prempaid_gi,lapse_ape_ltc_1280bf,lapse_ape_grp_6fc3e6,lapse_ape_grp_de05ae,lapse_ape_inv_dcd836,lapse_ape_grp_945b5a,lapse_ape_grp_6a5788,lapse_ape_ltc_43b9d5,lapse_ape_grp_9cdedf,lapse_ape_lh_d0adeb,lapse_ape_grp_1581d7,lapse_ape_grp_22decf,lapse_ape_lh_507c37,lapse_ape_lh_839f8a,lapse_ape_inv_e9f316,lapse_ape_grp_caa6ff,lapse_ape_grp_fd3bfb,lapse_ape_lh_e22a6a,lapse_ape_grp_70e1dd,lapse_ape_grp_e04c3a,lapse_ape_grp_fe5fb8,lapse_ape_grp_94baec,lapse_ape_grp_e91421,lapse_ape_lh_f852af,lapse_ape_lh_947b15,lapse_ape_32c74c,n_months_since_lapse_ltc_1280bf,n_months_since_lapse_grp_6fc3e6,n_months_since_lapse_grp_de05ae,n_months_since_lapse_inv_dcd836,n_months_since_lapse_grp_945b5a,n_months_since_lapse_grp_6a5788,n_months_since_lapse_ltc_43b9d5,n_months_since_lapse_grp_9cdedf,n_months_since_lapse_lh_d0adeb,n_months_since_lapse_grp_1581d7,n_months_since_lapse_grp_22decf,n_months_since_lapse_lh_507c37,n_months_since_lapse_lh_839f8a,n_months_since_lapse_inv_e9f316,n_months_since_lapse_grp_caa6ff,n_months_since_lapse_grp_fd3bfb,n_months_since_lapse_lh_e22a6a,n_months_since_lapse_grp_70e1dd,n_months_since_lapse_grp_e04c3a,n_months_since_lapse_grp_fe5fb8,n_months_since_lapse_grp_94baec,n_months_since_lapse_grp_e91421,n_months_since_lapse_lh_f852af,n_months_since_lapse_lh_947b15,n_months_since_lapse_32c74c,f_ever_bought_839f8a,f_ever_bought_e22a6a,f_ever_bought_d0adeb,f_ever_bought_c4bda5,f_ever_bought_ltc,f_ever_bought_507c37,f_ever_bought_

In [347]:
def clean_general_info(df):
    df1 = df.copy()

    # drop clntnum
    df1 = df1.drop("clntnum", axis=1)

    # question: do we need to combine race or country?
    df1['race_desc'] = df1['race_desc'].fillna("Others")

    # Question: Need to keep both Not applicable and unknown country code?
    df1['ctrycode_desc'] = df1['ctrycode_desc'].fillna("Not Applicable")

    df1['ctrycode_desc'] = df1['ctrycode_desc'].apply(lambda x: x if x == 'Singapore' else 'Others')

    # convert to datetime 
    # check nan value, should we fill it with today?
    df1['min_occ_date'] = pd.to_datetime(df1['min_occ_date'], errors='coerce')
    years_with_singlife = (pd.to_datetime('now') - df1['min_occ_date']).dt.days / 365.25
    mean_duration = years_with_singlife.median() # pick median instead of mean ???
    df1['years_with_company'] = years_with_singlife.fillna(mean_duration).astype(int)
    df1.drop('min_occ_date', axis=1, inplace=True) # drop??

    # cltdob_fix: calculate age
    df1['cltdob_fix'] = pd.to_datetime(df1['cltdob_fix'], errors='coerce')
    age_in_years = (pd.to_datetime('now') - df1['cltdob_fix']).dt.days / 365.25
    mean_age = age_in_years.mean()
    df1['age'] = age_in_years.fillna(mean_age).astype(int) # fill with mean age
    # dropping DOB column 
    df1.drop('cltdob_fix', axis=1, inplace=True)

    # None value in gender 
    df1['cltsex_fix'].unique()
    # method 1: if None then male, method2: dropna, here I used method 1
    df1['cltsex_fix'] = df1['cltsex_fix'].fillna('Male')

    return df1 
df1 = clean_general_info(df)
df1.head().style

,race_desc,ctrycode_desc,clttype,stat_flag,cltsex_fix,flg_substandard,flg_is_borderline_standard,flg_is_revised_term,flg_is_rental_flat,flg_has_health_claim,flg_has_life_claim,flg_gi_claim,flg_is_proposal,flg_with_preauthorisation,flg_is_returned_mail,is_consent_to_mail,is_consent_to_email,is_consent_to_call,is_consent_to_sms,is_valid_dm,is_valid_email,is_housewife_retiree,is_sg_pr,is_class_1_2,is_dependent_in_at_least_1_policy,f_ever_declined_la,hh_20,pop_20,hh_size,hh_size_est,annual_income_est,n_months_last_bought_products,flg_latest_being_lapse,flg_latest_being_cancel,recency_lapse,recency_cancel,tot_inforce_pols,tot_cancel_pols,ape_gi_42e115,ape_ltc_1280bf,ape_grp_6fc3e6,ape_grp_de05ae,ape_inv_dcd836,ape_grp_945b5a,ape_grp_6a5788,ape_ltc_43b9d5,ape_grp_9cdedf,ape_lh_d0adeb,ape_grp_1581d7,ape_grp_22decf,ape_lh_507c37,ape_lh_839f8a,ape_inv_e9f316,ape_gi_a10d1b,ape_gi_29d435,ape_grp_caa6ff,ape_grp_fd3bfb,ape_lh_e22a6a,ape_grp_70e1dd,ape_grp_e04c3a,ape_grp_fe5fb8,ape_gi_856320,ape_grp_94baec,ape_gi_058815,ape_grp_e91421,ape_lh_f852af,ape_lh_947b15,ape_32c74c,sumins_gi_42e115,sumins_ltc_1280bf,sumins_grp_6fc3e6,sumins_grp_de05ae,sumins_inv_dcd836,sumins_grp_945b5a,sumins_grp_6a5788,sumins_ltc_43b9d5,sumins_grp_9cdedf,sumins_lh_d0adeb,sumins_grp_1581d7,sumins_grp_22decf,sumins_lh_507c37,sumins_inv_e9f316,sumins_gi_a10d1b,sumins_gi_29d435,sumins_grp_caa6ff,sumins_grp_fd3bfb,sumins_lh_e22a6a,sumins_grp_70e1dd,sumins_grp_e04c3a,sumins_grp_fe5fb8,sumins_gi_856320,sumins_grp_94baec,sumins_gi_058815,sumins_grp_e91421,sumins_lh_f852af,sumins_lh_947b15,sumins_32c74c,prempaid_gi_42e115,prempaid_ltc_1280bf,prempaid_grp_6fc3e6,prempaid_grp_de05ae,prempaid_inv_dcd836,prempaid_grp_945b5a,prempaid_grp_6a5788,prempaid_ltc_43b9d5,prempaid_grp_9cdedf,prempaid_lh_d0adeb,prempaid_grp_1581d7,prempaid_grp_22decf,prempaid_lh_507c37,prempaid_lh_839f8a,prempaid_inv_e9f316,prempaid_gi_a10d1b,prempaid_gi_29d435,prempaid_grp_caa6ff,prempaid_grp_fd3bfb,prempaid_lh_e22a6a,prempaid_grp_70e1dd,prempaid_grp_e04c3a,prempaid_grp_fe5fb8,prempaid_gi_856320,prempaid_grp_94baec,prempaid_gi_058815,prempaid_grp_e91421,prempaid_lh_f852af,prempaid_lh_947b15,prempaid_32c74c,ape_839f8a,ape_e22a6a,ape_d0adeb,ape_c4bda5,ape_ltc,ape_507c37,ape_gi,f_hold_839f8a,f_hold_e22a6a,f_hold_d0adeb,f_hold_c4bda5,f_hold_ltc,f_hold_507c37,f_hold_gi,sumins_839f8a,sumins_e22a6a,sumins_d0adeb,sumins_c4bda5,sumins_ltc,sumins_507c37,sumins_gi,prempaid_839f8a,prempaid_e22a6a,prempaid_d0adeb,prempaid_c4bda5,prempaid_ltc,prempaid_507c37,prempaid_gi,lapse_ape_ltc_1280bf,lapse_ape_grp_6fc3e6,lapse_ape_grp_de05ae,lapse_ape_inv_dcd836,lapse_ape_grp_945b5a,lapse_ape_grp_6a5788,lapse_ape_ltc_43b9d5,lapse_ape_grp_9cdedf,lapse_ape_lh_d0adeb,lapse_ape_grp_1581d7,lapse_ape_grp_22decf,lapse_ape_lh_507c37,lapse_ape_lh_839f8a,lapse_ape_inv_e9f316,lapse_ape_grp_caa6ff,lapse_ape_grp_fd3bfb,lapse_ape_lh_e22a6a,lapse_ape_grp_70e1dd,lapse_ape_grp_e04c3a,lapse_ape_grp_fe5fb8,lapse_ape_grp_94baec,lapse_ape_grp_e91421,lapse_ape_lh_f852af,lapse_ape_lh_947b15,lapse_ape_32c74c,n_months_since_lapse_ltc_1280bf,n_months_since_lapse_grp_6fc3e6,n_months_since_lapse_grp_de05ae,n_months_since_lapse_inv_dcd836,n_months_since_lapse_grp_945b5a,n_months_since_lapse_grp_6a5788,n_months_since_lapse_ltc_43b9d5,n_months_since_lapse_grp_9cdedf,n_months_since_lapse_lh_d0adeb,n_months_since_lapse_grp_1581d7,n_months_since_lapse_grp_22decf,n_months_since_lapse_lh_507c37,n_months_since_lapse_lh_839f8a,n_months_since_lapse_inv_e9f316,n_months_since_lapse_grp_caa6ff,n_months_since_lapse_grp_fd3bfb,n_months_since_lapse_lh_e22a6a,n_months_since_lapse_grp_70e1dd,n_months_since_lapse_grp_e04c3a,n_months_since_lapse_grp_fe5fb8,n_months_since_lapse_grp_94baec,n_months_since_lapse_grp_e91421,n_months_since_lapse_lh_f852af,n_months_since_lapse_lh_947b15,n_months_since_lapse_32c74c,f_ever_bought_839f8a,f_ever_bought_e22a6a,f_ever_bought_d0adeb,f_ever_bought_c4bda5,f_ever_bought_ltc,f_ever_bought_507c37,f_ever_bought_gi,n_months_last_bought_839f8a,n

In [348]:
df1['num_total_claims'] = (df1['flg_has_health_claim'] + df1['flg_has_life_claim'] + df1['flg_gi_claim']).fillna(0)

In [349]:
df1.shape

(17992, 304)

In [350]:
# 3. combine risk together: substandard risk -> level 3, boarderline -> level 2, standard -> level 1 
conditions = [
    df1['flg_substandard'] == 1,
    df1['flg_is_borderline_standard'] == 1,
    ~(df1['flg_substandard'] == 1) & ~(df1['flg_is_borderline_standard'] == 1)  # Standard risk
]

values = [3, 2, 1]

# Create a new column 'risk_level' based on the conditions, if na means standard 
df1['risk_level'] = np.select(conditions, values, default=1)

df1.drop(columns = ['flg_substandard','flg_is_borderline_standard','flg_has_health_claim','flg_has_life_claim','flg_gi_claim'],inplace=True)


In [351]:
df1.shape

(17992, 300)

In [352]:
df1.iloc[:,18:22].head().style

,is_class_1_2,is_dependent_in_at_least_1_policy,f_ever_declined_la,hh_20
19550,1.000000,0.000000,nan,144
4600,0.000000,0.000000,nan,153
13337,1.000000,0.000000,nan,62
15074,1.000000,0.000000,nan,1
19724,1.000000,0.000000,nan,114


In [353]:
# convert to boolean: is_housewife_retiree, is_sg_pr, is_class_1_2, is_dependent_in_at_least_1_policy 
col = ['is_housewife_retiree', 'is_sg_pr', 'is_class_1_2', 'is_dependent_in_at_least_1_policy']
for column in col:
    df1[column] = df1[column].fillna(0).astype(float).astype(int)

In [354]:
df1['annual_income_est'] = df1['annual_income_est'].fillna("F.UNKNOWN")
income_mapping = {
    'F. OTHERS': 1,
    'E.BELOW30K': 2,
    'D.30K-60K': 3,
    'C.60K-100K': 4,
    'B.100K-200K': 5,
    'A.ABOVE200K': 6
}

# Apply ordinal encoding
df1['annual_income_encoded'] = df1['annual_income_est'].map(income_mapping)

In [355]:
df1['hh_size_est'] = df1['hh_size_est'].fillna("0")

hh_size_mapping = {
    '1': 1,
    '2': 2,
    '3': 3,
    '4': 4,
    '>4': 5,
    '0': 0
}

# Apply ordinal encoding
df1['hh_size_est_encoded'] = df1['hh_size_est'].map(hh_size_mapping)

df1.drop(columns=['hh_20','pop_20','hh_size','hh_size_est','annual_income_est'],inplace=True)

In [356]:
consent_col = ['is_consent_to_call', 'is_consent_to_email', 'is_consent_to_mail', 'is_consent_to_sms']

for column in consent_col:
    df1[column] = df1[column].fillna(0).astype(float).astype(int) # na can just assume they don't consent 

conditions = [
    (df1['is_consent_to_call'] == 1),
    (df1['is_consent_to_email'] == 1) & (df1['is_valid_email'] == 1),
    (df1['is_consent_to_mail'] == 1) & (df1['is_valid_dm'] == 1),
    (df1['is_consent_to_sms'] == 1)
]

values = [5, 1, 1, 1]

# Create the 'consent_level' column by summing the values based on the conditions
df1['consent_level'] = np.sum(np.column_stack(conditions) * np.array(values), axis=1)
df1.drop(columns=['is_consent_to_mail','is_consent_to_email','is_consent_to_call','is_consent_to_sms','is_valid_dm','is_valid_email'],inplace=True)

In [357]:
df1['f_ever_declined_la'] = df1['f_ever_declined_la'].fillna(0)

# 延迟时间
df1["recency_lapse"] = df1["recency_lapse"].fillna(0)
df1["recency_cancel"] = df1["recency_cancel"].fillna(0)

df1 = df1.drop(columns=['flg_latest_being_lapse', 'flg_latest_being_cancel'], axis=1)

df1['tot_cancel_pols'] = df1['tot_cancel_pols'].fillna(0)

# df1['avg_purchase_frequency'] = (df1['tot_cancel_pols'] + df1['tot_inforce_pols']) / df1['years_with_company'] # division by 0????
# df1['avg_cancel_rate'] = df1['tot_cancel_pols'] / (df1['tot_cancel_pols'] + df1['tot_inforce_pols']) 

In [358]:
df1.shape

(17992, 290)

In [359]:
last_bought_months = df1.filter(regex='n_months_last_bought_.*', axis=1).columns

for column in last_bought_months:
    df1[column] = df1[column].fillna(0).astype(float).astype(int)


df1[last_bought_months] = df1[last_bought_months].replace(9999, 0)

df1['n_months_last_bought_products'] = df1['n_months_last_bought_products'].fillna(0)

df1['Combined_List_Bought'] = df1[last_bought_months].apply(lambda row: sorted([val for val in row.tolist() if val != 0]), axis=1)
df1['Bought_Gap_List'] = df1['Combined_List_Bought'].apply(lambda sorted_list: [b - a for a, b in zip(sorted_list[:-1], sorted_list[1:])])
df1['Average_Bought_Gap'] = df1['Bought_Gap_List'].apply(lambda gaps: np.mean(gaps) if gaps else 0)

df1['Average_Bought_Gap'][df1['Average_Bought_Gap'] !=0]

df1['Exceed Average Gap'] = df1['Average_Bought_Gap'] <= df1['n_months_last_bought_products']
sum(df1['Exceed Average Gap'])

df1['Exceed Average Gap with Purchase'] = (df1['Average_Bought_Gap'] <= df1['n_months_last_bought_products']) & (df1['Average_Bought_Gap'] != 0)
sum(df1['Exceed Average Gap with Purchase'])

df1['Average_Bought_Gap'][df1['Average_Bought_Gap']>=36]

bins = [-float('inf'), 3, 6, 12, 24, 36, 60, 120, float('inf')]
labels = ['<3 months', '3-6 months', '6 months - 1 year', '1-2 years', '2-3 years', '3-5 years', '5-10 years', '> 10 years']

df1['Months_Last_Bought_Category'] = pd.cut(df1['n_months_last_bought_products'], bins=bins, labels=labels)

df1['Months_Last_Bought_Category']

column_patterns = ['ape_', 'sumins_', 'prempaid_']
regex_pattern = '^(' + '|'.join(column_patterns) + ')'
claimed_amount = df1.filter(regex=regex_pattern, axis=1).columns

df1['Claimed_Large_Amount'] = df1[claimed_amount].ge(50000).any(axis=1)
sum(df1['Claimed_Large_Amount'])

column_patterns = ['lapse_']
regex_pattern = '^(' + '|'.join(column_patterns) + ')'
lapsed_amount = df1.filter(regex=regex_pattern, axis=1).columns

df1['Lapsed_Large_Amount'] = df1[lapsed_amount].ge(200).any(axis=1)
sum(df1['Lapsed_Large_Amount'])

last_lapse_months =  df1.filter(regex='n_months_since_lapse.*', axis=1).columns


#df1[last_lapse_months] = df1[last_lapse_months].replace('9999', 0)
for column in last_lapse_months:
    df1[column] = df1[column].fillna(0).astype(float).astype(int)


df1[last_lapse_months] = df1[last_lapse_months].replace(9999, 0)


df1['Combined_List_Lapse'] = df1[last_lapse_months].apply(lambda row: sorted([val for val in row.tolist() if val != 0]), axis=1)
df1['Lapse_Gap_List'] = df1['Combined_List_Lapse'].apply(lambda sorted_list: [b - a for a, b in zip(sorted_list[:-1], sorted_list[1:])])
df1['Average_Lapse_Gap'] = df1['Lapse_Gap_List'].apply(lambda gaps: np.mean(gaps) if gaps else 0)

columns_to_drop = []

column_patterns = ['f_ever_bought']
regex_pattern = '^(' + '|'.join(column_patterns) + ')'
ever_bought = df1.filter(regex=regex_pattern, axis=1).columns

columns_to_drop.extend(last_bought_months)
columns_to_drop.extend(claimed_amount)
columns_to_drop.extend(last_lapse_months)
columns_to_drop.extend(lapsed_amount)
columns_to_drop.extend(ever_bought)
columns_to_drop.extend(['Combined_List_Lapse','Lapse_Gap_List','Combined_List_Bought', 'Bought_Gap_List'])
df1.drop(columns=columns_to_drop, inplace=True)

In [360]:
f_hold_columns = df1.filter(regex='f_hold_.*').columns

for col in f_hold_columns:
    df1[col] = df1[col].astype(int)

df1["f_hold"] = df1[f_hold_columns].sum(axis=1)
df1.drop(columns=f_hold_columns, inplace=True)


In [361]:
print(df1.shape)
df1.head().style

(17992, 66)


,race_desc,ctrycode_desc,clttype,stat_flag,cltsex_fix,flg_is_revised_term,flg_is_rental_flat,flg_is_proposal,flg_with_preauthorisation,flg_is_returned_mail,is_housewife_retiree,is_sg_pr,is_class_1_2,is_dependent_in_at_least_1_policy,f_ever_declined_la,recency_lapse,recency_cancel,tot_inforce_pols,tot_cancel_pols,f_elx,f_mindef_mha,f_retail,flg_affconnect_show_interest_ever,flg_affconnect_ready_to_buy_ever,flg_affconnect_lapse_ever,affcon_visit_days,n_months_since_visit_affcon,clmcon_visit_days,recency_clmcon,recency_clmcon_regis,hlthclaim_amt,recency_hlthclaim,hlthclaim_cnt_success,recency_hlthclaim_success,hlthclaim_cnt_unsuccess,recency_hlthclaim_unsuccess,flg_hlthclaim_839f8a_ever,recency_hlthclaim_839f8a,flg_hlthclaim_14cb37_ever,recency_hlthclaim_14cb37,giclaim_amt,recency_giclaim,giclaim_cnt_success,recency_giclaim_success,giclaim_cnt_unsuccess,recency_giclaim_unsuccess,flg_gi_claim_29d435_ever,flg_gi_claim_058815_ever,flg_gi_claim_42e115_ever,flg_gi_claim_856320_ever,f_purchase_lh,years_with_company,age,num_total_claims,risk_level,annual_income_encoded,hh_size_est_encoded,consent_level,Average_Bought_Gap,Exceed Average Gap,Exceed Average Gap with Purchase,Months_Last_Bought_Category,Claimed_Large_Amount,Lapsed_Large_Amount,Average_Lapse_Gap,f_hold
19550,Chinese,Singapore,P,ACTIVE,Female,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,1,0,0.000000,29.000000,0.000000,3,0.000000,0,0,1,nan,nan,nan,nan,nan,nan,nan,nan,None,nan,nan,nan,nan,nan,nan,nan,nan,nan,None,nan,None,None,None,None,None,None,None,None,nan,6,49,0.000000,1,4.000000,1,0,1.250000,False,False,<3 months,False,False,0.000000,2
4600,Chinese,Singapore,P,ACTIVE,Male,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,0,0,0.000000,140.000000,0.000000,1,0.000000,0,0,1,nan,nan,nan,nan,nan,nan,nan,nan,None,nan,nan,nan,nan,nan,nan,nan,nan,nan,None,nan,None,None,None,None,None,None,None,None,nan,16,44,0.000000,1,3.000000,3,0,47.500000,False,False,3-5 years,False,True,0.000000,0
13337,Others,Singapore,P,ACTIVE,Male,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,1,0,0.000000,0.000000,0.000000,1,0.000000,0,0,1,nan,nan,nan,nan,nan,nan,nan,nan,None,nan,nan,nan,nan,nan,nan,nan,nan,nan,None,nan,None,None,None,None,None,None,None,None,nan,4,48,0.000000,1,6.000000,3,2,0.000000,True,False,3-5 years,False,False,0.000000,0
15074,Chinese,Singapore,P,ACTIVE,Female,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,1,0,0.000000,0.000000,0.000000,1,0.000000,0,0,1,nan,nan,nan,nan,nan,nan,nan,nan,None,nan,nan,nan,nan,nan,nan,nan,nan,nan,None,nan,None,None,None,None,None,None,None,None,nan,2,47,0.000000,1,5.000000,4,0,0.000000,True,False,1-2 years,False,False,0.000000,0
19724,Chinese,Singapore,P,ACTIVE,Female,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,1,0,0.000000,0.000000,0.000000,1,0.000000,0,0,1,nan,nan,nan,nan,nan,nan,nan,nan,None,nan,nan,nan,nan,nan,nan,nan,nan,nan,None,nan,None,None,None,None,None,None,None,None,nan,5,28,0.000000,1,2.000000,5,3,0.000000,True,False,5-10 years,True,False,0.000000,1


In [363]:
import pandas as pd
pattern = r'flg_(hlthclaim|gi_claim)_[a-z0-9]+_ever'
filtered_cols = df.filter(regex=pattern).columns

print(filtered_cols)

df1[filtered_cols] = df1[filtered_cols].fillna(0)
df1['flg_hlthclaim_ever'] = df1[filtered_cols].sum(axis=1)

df1.drop(columns=filtered_cols, inplace=True)


Index(['flg_hlthclaim_839f8a_ever', 'flg_hlthclaim_14cb37_ever',
       'flg_gi_claim_29d435_ever', 'flg_gi_claim_058815_ever',
       'flg_gi_claim_42e115_ever', 'flg_gi_claim_856320_ever'],
      dtype='object')


In [364]:
print(df1.shape)
df1.head().style

(17992, 61)


,race_desc,ctrycode_desc,clttype,stat_flag,cltsex_fix,flg_is_revised_term,flg_is_rental_flat,flg_is_proposal,flg_with_preauthorisation,flg_is_returned_mail,is_housewife_retiree,is_sg_pr,is_class_1_2,is_dependent_in_at_least_1_policy,f_ever_declined_la,recency_lapse,recency_cancel,tot_inforce_pols,tot_cancel_pols,f_elx,f_mindef_mha,f_retail,flg_affconnect_show_interest_ever,flg_affconnect_ready_to_buy_ever,flg_affconnect_lapse_ever,affcon_visit_days,n_months_since_visit_affcon,clmcon_visit_days,recency_clmcon,recency_clmcon_regis,hlthclaim_amt,recency_hlthclaim,hlthclaim_cnt_success,recency_hlthclaim_success,hlthclaim_cnt_unsuccess,recency_hlthclaim_unsuccess,recency_hlthclaim_839f8a,recency_hlthclaim_14cb37,giclaim_amt,recency_giclaim,giclaim_cnt_success,recency_giclaim_success,giclaim_cnt_unsuccess,recency_giclaim_unsuccess,f_purchase_lh,years_with_company,age,num_total_claims,risk_level,annual_income_encoded,hh_size_est_encoded,consent_level,Average_Bought_Gap,Exceed Average Gap,Exceed Average Gap with Purchase,Months_Last_Bought_Category,Claimed_Large_Amount,Lapsed_Large_Amount,Average_Lapse_Gap,f_hold,flg_hlthclaim_ever
19550,Chinese,Singapore,P,ACTIVE,Female,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,1,0,0.000000,29.000000,0.000000,3,0.000000,0,0,1,nan,nan,nan,nan,nan,nan,nan,nan,None,nan,nan,nan,nan,nan,nan,nan,None,nan,None,None,None,None,nan,6,49,0.000000,1,4.000000,1,0,1.250000,False,False,<3 months,False,False,0.000000,2,0.000000
4600,Chinese,Singapore,P,ACTIVE,Male,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,0,0,0.000000,140.000000,0.000000,1,0.000000,0,0,1,nan,nan,nan,nan,nan,nan,nan,nan,None,nan,nan,nan,nan,nan,nan,nan,None,nan,None,None,None,None,nan,16,44,0.000000,1,3.000000,3,0,47.500000,False,False,3-5 years,False,True,0.000000,0,0.000000
13337,Others,Singapore,P,ACTIVE,Male,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,1,0,0.000000,0.000000,0.000000,1,0.000000,0,0,1,nan,nan,nan,nan,nan,nan,nan,nan,None,nan,nan,nan,nan,nan,nan,nan,None,nan,None,None,None,None,nan,4,48,0.000000,1,6.000000,3,2,0.000000,True,False,3-5 years,False,False,0.000000,0,0.000000
15074,Chinese,Singapore,P,ACTIVE,Female,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,1,0,0.000000,0.000000,0.000000,1,0.000000,0,0,1,nan,nan,nan,nan,nan,nan,nan,nan,None,nan,nan,nan,nan,nan,nan,nan,None,nan,None,None,None,None,nan,2,47,0.000000,1,5.000000,4,0,0.000000,True,False,1-2 years,False,False,0.000000,0,0.000000
19724,Chinese,Singapore,P,ACTIVE,Female,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,1,0,0.000000,0.000000,0.000000,1,0.000000,0,0,1,nan,nan,nan,nan,nan,nan,nan,nan,None,nan,nan,nan,nan,nan,nan,nan,None,nan,None,None,None,None,nan,5,28,0.000000,1,2.000000,5,3,0.000000,True,False,5-10 years,True,False,0.000000,1,0.000000


In [417]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline
from sklearn.metrics import f1_score

# Assuming df is your DataFrame and 'f_purchase_lh' is the target column
df = df1
col_to_target = ['f_purchase_lh']
for col in col_to_target:
    df1[col] = df1[col].fillna(0)
# Separate features and target
X = df.drop('f_purchase_lh', axis=1)
y = df['f_purchase_lh']

# Separate features and target
X = df.drop('f_purchase_lh', axis=1)
y = df['f_purchase_lh']

# Identify categorical and numerical columns
categorical_cols = X.select_dtypes(include=['object', 'category']).columns
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns

# Create a preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', SimpleImputer(strategy='mean'), numerical_cols),
        ('cat', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), categorical_cols)
    ])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a pipeline with preprocessing, SMOTE, and LGBM Classifier
model = Pipeline([
    ('preprocessor', preprocessor),
    ('scaler', StandardScaler()),
    ('smote', SMOTE(random_state=42)),
    ('classifier', LGBMClassifier(random_state=42))
])

# Train the model
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Evaluation
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

f1 = f1_score(y_test, y_pred)

print("F1 Score: ", f1)


[LightGBM] [Info] Number of positive: 13834, number of negative: 13834
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004323 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 10139
[LightGBM] [Info] Number of data points in the train set: 27668, number of used features: 48
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[[3438   10]
 [ 143    8]]
              precision    recall  f1-score   support

         0.0       0.96      1.00      0.98      3448
         1.0       0.44      0.05      0.09       151

    accuracy                           0.96      3599
   macro avg       0.70      0.53      0.54      3599
weighted avg       0.94      0.96      0.94      3599

F1 Score:  0.09467455621301776


In [415]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from sklearn.metrics import f1_score

# Assuming df is your DataFrame and 'f_purchase_lh' is the target column
df = df1
col_to_target = ['f_purchase_lh']
for col in col_to_target:
    df1[col] = df1[col].fillna(0)
# Separate features and target
X = df.drop('f_purchase_lh', axis=1)
y = df['f_purchase_lh']

# Identify categorical and numerical columns
categorical_cols = X.select_dtypes(include=['object', 'category']).columns
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns

preprocessor = ColumnTransformer(
    transformers=[
        ('num', SimpleImputer(strategy='mean'), numerical_cols),
        ('cat', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), categorical_cols)
    ])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a pipeline with preprocessing, SMOTE, and Random Forest Classifier
model = Pipeline([
    ('preprocessor', preprocessor),
    ('scaler', StandardScaler()),
    ('smote', SMOTE(random_state=42)),
    ('classifier', RandomForestClassifier(random_state=42))
])

# Train the model
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Evaluation
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

f1 = f1_score(y_test, y_pred)

print("F1 Score: ", f1)


[[3429   19]
 [ 144    7]]
              precision    recall  f1-score   support

         0.0       0.96      0.99      0.98      3448
         1.0       0.27      0.05      0.08       151

    accuracy                           0.95      3599
   macro avg       0.61      0.52      0.53      3599
weighted avg       0.93      0.95      0.94      3599

F1 Score:  0.0790960451977401


In [412]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from sklearn.metrics import f1_score

# Assuming df is your DataFrame and 'f_purchase_lh' is the target column

df = df1
col_to_target = ['f_purchase_lh']
for col in col_to_target:
    df1[col] = df1[col].fillna(0)

# Separate features and target
X = df.drop('f_purchase_lh', axis=1)
y = df['f_purchase_lh']

# Identify categorical and numerical columns
categorical_cols = X.select_dtypes(include=['object', 'category']).columns
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns

# Create a preprocessing pipeline
# Inside your ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', SimpleImputer(strategy='mean'), numerical_cols),
        ('cat', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), categorical_cols)
    ])


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a pipeline with preprocessing, SMOTE, and logistic regression
model = Pipeline([
    ('preprocessor', preprocessor),
    ('scaler', StandardScaler()),
    ('smote', SMOTE(random_state=42)),
    ('classifier', LogisticRegression(random_state=42))
])

# Train the model
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Evaluation
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

f1 = f1_score(y_test, y_pred)

print("F1 Score: ", f1)

[[2478  970]
 [  43  108]]
              precision    recall  f1-score   support

         0.0       0.98      0.72      0.83      3448
         1.0       0.10      0.72      0.18       151

    accuracy                           0.72      3599
   macro avg       0.54      0.72      0.50      3599
weighted avg       0.95      0.72      0.80      3599

F1 Score:  0.1757526444263629


In [410]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim
from datetime import datetime
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Your existing code for data preparation
col_to_target = ['f_purchase_lh']
for col in col_to_target:
    df1[col] = df1[col].fillna(0)

data = df1
print(data.shape)

X = data.drop(['f_purchase_lh'], axis=1)
y = data['f_purchase_lh']

categorical_cols = X.select_dtypes(include=['object', 'category']).columns
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns

# Modified preprocessor with OrdinalEncoder
preprocessor = ColumnTransformer(
    transformers=[
        ('num', SimpleImputer(strategy='mean'), numerical_cols),
        ('cat', OrdinalEncoder(), categorical_cols)  # Using OrdinalEncoder for categorical features
    ])

X = preprocessor.fit_transform(X)
print(X.shape)

# Continue with your scaling and further processing
scaler = StandardScaler(with_mean=False)
X = scaler.fit_transform(X)
X = np.asarray(X)
print(X.shape)

# No change in these parts of the code
X_tensor = torch.tensor(X.astype(np.float32))
y_tensor = torch.tensor(y.values.astype(np.float32))

X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)


(17992, 64)
(17992, 59)
(17992, 59)
F1 Score:  0.1757526444263629


In [402]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.layer1 = nn.Linear(X_train.shape[1], 256)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(0.5)
        self.layer2 = nn.Linear(256, 128)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(0.5)
        self.layer3 = nn.Linear(128, 64)
        self.relu3 = nn.ReLU()
        self.output_layer = nn.Linear(64, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.dropout1(self.relu1(self.layer1(x)))
        x = self.dropout2(self.relu2(self.layer2(x)))
        x = self.relu3(self.layer3(x))
        x = self.sigmoid(self.output_layer(x))
        return x

model = NeuralNetwork()


In [397]:
import torch.nn.functional as F
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(X_train.shape[1], 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x

In [403]:
class_counts = data['f_purchase_lh'].value_counts().sort_index().values
total_samples = class_counts.sum()
weights = total_samples / torch.tensor(class_counts, dtype=torch.float32)

# If using GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
weights = weights.to(device)
model.to(device)

# Modify the BCELoss to handle weights for each batch
def weighted_bce_loss(outputs, targets):
    loss = nn.BCELoss(reduction='none')(outputs, targets)
    weighted_loss = loss * weights[targets.long()]
    return weighted_loss.mean()

optimizer = optim.Adam(model.parameters(), lr=0.00005)

train_data = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)

for epoch in range(30): 
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = weighted_bce_loss(outputs, labels.unsqueeze(1))
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item()}")


Epoch 1, Loss: 1.6231539249420166
Epoch 2, Loss: 1.6342189311981201
Epoch 3, Loss: 0.915778398513794
Epoch 4, Loss: 1.467997670173645
Epoch 5, Loss: 1.2895747423171997
Epoch 6, Loss: 1.1745104789733887
Epoch 7, Loss: 1.6579558849334717
Epoch 8, Loss: 0.6961671710014343
Epoch 9, Loss: 1.8773548603057861
Epoch 10, Loss: 1.3884607553482056
Epoch 11, Loss: 1.2910796403884888
Epoch 12, Loss: 0.8429020643234253
Epoch 13, Loss: 0.7205491065979004
Epoch 14, Loss: 1.3329249620437622
Epoch 15, Loss: 1.067515254020691
Epoch 16, Loss: 1.0352634191513062
Epoch 17, Loss: 1.1536399126052856
Epoch 18, Loss: 0.5768652558326721
Epoch 19, Loss: 0.837746798992157
Epoch 20, Loss: 1.2716888189315796
Epoch 21, Loss: 1.2224128246307373
Epoch 22, Loss: 0.9818214178085327
Epoch 23, Loss: 1.2102051973342896
Epoch 24, Loss: 1.4704692363739014
Epoch 25, Loss: 0.6934013366699219
Epoch 26, Loss: 0.7820970416069031
Epoch 27, Loss: 0.6434139013290405
Epoch 28, Loss: 0.9204712510108948
Epoch 29, Loss: 1.266822099685669

In [404]:
test_data = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        predicted = (outputs > 0.5).float()
        total += labels.size(0)
        correct += (predicted.squeeze() == labels).sum().item()

print(f'Accuracy: {100 * correct / total}%')


Accuracy: 74.29841622672964%


In [405]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Function to calculate precision, recall, and F1 score
def calculate_metrics(y_true, y_pred):
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    return precision, recall, f1

# Convert test data to DataLoader
test_data = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

# Evaluate the model
model.eval()
y_pred = []
y_true = []
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        predicted = (outputs > 0.5).float()
        y_pred.extend(predicted.squeeze().tolist())
        y_true.extend(labels.tolist())

# Calculate metrics
precision, recall, f1 = calculate_metrics(np.array(y_true), np.array(y_pred))
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')


Precision: 0.10429447852760736
Recall: 0.6754966887417219
F1 Score: 0.18069087688219665


# 下面的别看了，我在试验

In [365]:
ever_bought_columns = df1.filter(regex='f_ever_bought_.*')
# n_months_last_bought_columns = df1.filter(regex='n_months_last_bought_.*').drop(columns=['n_months_last_bought_products'])
n_months_last_bought_columns = df1.filter(regex='n_months_last_bought_.*')

In [366]:
df1 = df1.drop(ever_bought_columns, axis=1)
# df1 = df1.drop(n_months_last_bought_columns, axis=1)
for col in n_months_last_bought_columns:
    df[col] = df[col].replace(9999, 0)

In [367]:
# premium prepaid 
def sum_and_drop_columns(df, prefix, new_column_name):
    # Filter columns based on the specified prefix
    filtered_columns = df.filter(regex=f'^{prefix}', axis=1)
    
    # Get the column names
    sumins_columns = filtered_columns.columns
    
    # Calculate the sum along the horizontal axis and create a new column
    df[new_column_name] = filtered_columns.sum(axis=1)
    
    # Drop the original columns
    df.drop(columns=sumins_columns, inplace=True)

# ape 
sum_and_drop_columns(df1, 'ape', 'total_ape_purchased')
# sum_and_drop_columns(df1, 'ape_gi', 'total_ape_gi_purchased')
# sum_and_drop_columns(df1, 'ape_grp', 'total_ape_grp_purchased')
# sum_and_drop_columns(df1, 'ape_inv', 'total_ape_inv_purchased')
# sum_and_drop_columns(df1, 'ape_lh', 'total_ape_lh_purchased')
# sum_and_drop_columns(df1, 'ape_ltc', 'total_ape_ltc_purchased')
# sum_and_drop_columns(df1, 'ape_', 'total_ape_others_purchased')

# sumins 
sum_and_drop_columns(df1, 'sumins', 'total_sumins_purchased')
# sum_and_drop_columns(df1, 'sumins_gi', 'total_sumins_gi_purchased')
# sum_and_drop_columns(df1, 'sumins_grp', 'total_sumins_grp_purchased')
# sum_and_drop_columns(df1, 'sumins_inv', 'total_sumins_inv_purchased')
# sum_and_drop_columns(df1, 'sumins_lh', 'total_sumins_lh_purchased')
# sum_and_drop_columns(df1, 'sumins_ltc', 'total_sumins_ltc_purchased')
# sum_and_drop_columns(df1, 'sumins_', 'total_sumins_others_purchased')

# prempad 
sum_and_drop_columns(df1, 'prempaid', 'total_prempaid_purchased')
# sum_and_drop_columns(df1, 'prempaid_gi', 'total_prempaid_gi_purchased')
# sum_and_drop_columns(df1, 'prempaid_grp', 'total_prempaid_grp_purchased')
# sum_and_drop_columns(df1, 'prempaid_inv', 'total_prempaid_inv_purchased')
# sum_and_drop_columns(df1, 'prempaid_lh', 'total_prempaid_lh_purchased')
# sum_and_drop_columns(df1, 'prempaid_ltc', 'total_prempaid_ltc_purchased')
# sum_and_drop_columns(df1, 'prempaid_', 'total_prempaid_others_purchased')

In [368]:
df1.head().style   

,race_desc,ctrycode_desc,clttype,stat_flag,cltsex_fix,flg_is_revised_term,flg_is_rental_flat,flg_is_proposal,flg_with_preauthorisation,flg_is_returned_mail,is_housewife_retiree,is_sg_pr,is_class_1_2,is_dependent_in_at_least_1_policy,f_ever_declined_la,recency_lapse,recency_cancel,tot_inforce_pols,tot_cancel_pols,f_elx,f_mindef_mha,f_retail,flg_affconnect_show_interest_ever,flg_affconnect_ready_to_buy_ever,flg_affconnect_lapse_ever,affcon_visit_days,n_months_since_visit_affcon,clmcon_visit_days,recency_clmcon,recency_clmcon_regis,hlthclaim_amt,recency_hlthclaim,hlthclaim_cnt_success,recency_hlthclaim_success,hlthclaim_cnt_unsuccess,recency_hlthclaim_unsuccess,recency_hlthclaim_839f8a,recency_hlthclaim_14cb37,giclaim_amt,recency_giclaim,giclaim_cnt_success,recency_giclaim_success,giclaim_cnt_unsuccess,recency_giclaim_unsuccess,f_purchase_lh,years_with_company,age,num_total_claims,risk_level,annual_income_encoded,hh_size_est_encoded,consent_level,Average_Bought_Gap,Exceed Average Gap,Exceed Average Gap with Purchase,Months_Last_Bought_Category,Claimed_Large_Amount,Lapsed_Large_Amount,Average_Lapse_Gap,f_hold,flg_hlthclaim_ever,total_ape_purchased,total_sumins_purchased,total_prempaid_purchased
19550,Chinese,Singapore,P,ACTIVE,Female,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,1,0,0.000000,29.000000,0.000000,3,0.000000,0,0,1,nan,nan,nan,nan,nan,nan,nan,nan,None,nan,nan,nan,nan,nan,nan,nan,None,nan,None,None,None,None,nan,6,49,0.000000,1,4.000000,1,0,1.250000,False,False,<3 months,False,False,0.000000,2,0.000000,0.000000,0.000000,0.000000
4600,Chinese,Singapore,P,ACTIVE,Male,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,0,0,0.000000,140.000000,0.000000,1,0.000000,0,0,1,nan,nan,nan,nan,nan,nan,nan,nan,None,nan,nan,nan,nan,nan,nan,nan,None,nan,None,None,None,None,nan,16,44,0.000000,1,3.000000,3,0,47.500000,False,False,3-5 years,False,True,0.000000,0,0.000000,0.000000,0.000000,0.000000
13337,Others,Singapore,P,ACTIVE,Male,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,1,0,0.000000,0.000000,0.000000,1,0.000000,0,0,1,nan,nan,nan,nan,nan,nan,nan,nan,None,nan,nan,nan,nan,nan,nan,nan,None,nan,None,None,None,None,nan,4,48,0.000000,1,6.000000,3,2,0.000000,True,False,3-5 years,False,False,0.000000,0,0.000000,0.000000,0.000000,0.000000
15074,Chinese,Singapore,P,ACTIVE,Female,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,1,0,0.000000,0.000000,0.000000,1,0.000000,0,0,1,nan,nan,nan,nan,nan,nan,nan,nan,None,nan,nan,nan,nan,nan,nan,nan,None,nan,None,None,None,None,nan,2,47,0.000000,1,5.000000,4,0,0.000000,True,False,1-2 years,False,False,0.000000,0,0.000000,0.000000,0.000000,0.000000
19724,Chinese,Singapore,P,ACTIVE,Female,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,1,0,0.000000,0.000000,0.000000,1,0.000000,0,0,1,nan,nan,nan,nan,nan,nan,nan,nan,None,nan,nan,nan,nan,nan,nan,nan,None,nan,None,None,None,None,nan,5,28,0.000000,1,2.000000,5,3,0.000000,True,False,5-10 years,True,False,0.000000,1,0.000000,0.000000,0.000000,0.000000


In [369]:
f_hold_columns = df1.filter(regex='f_hold_.*').columns

for col in f_hold_columns:
    df1[col] = df1[col].astype(int)

df1["f_hold"] = df1[f_hold_columns].sum(axis=1)
df1.drop(columns=f_hold_columns, inplace=True)


In [370]:
df1.head().style

,race_desc,ctrycode_desc,clttype,stat_flag,cltsex_fix,flg_is_revised_term,flg_is_rental_flat,flg_is_proposal,flg_with_preauthorisation,flg_is_returned_mail,is_housewife_retiree,is_sg_pr,is_class_1_2,is_dependent_in_at_least_1_policy,f_ever_declined_la,recency_lapse,recency_cancel,tot_inforce_pols,tot_cancel_pols,f_elx,f_mindef_mha,f_retail,flg_affconnect_show_interest_ever,flg_affconnect_ready_to_buy_ever,flg_affconnect_lapse_ever,affcon_visit_days,n_months_since_visit_affcon,clmcon_visit_days,recency_clmcon,recency_clmcon_regis,hlthclaim_amt,recency_hlthclaim,hlthclaim_cnt_success,recency_hlthclaim_success,hlthclaim_cnt_unsuccess,recency_hlthclaim_unsuccess,recency_hlthclaim_839f8a,recency_hlthclaim_14cb37,giclaim_amt,recency_giclaim,giclaim_cnt_success,recency_giclaim_success,giclaim_cnt_unsuccess,recency_giclaim_unsuccess,f_purchase_lh,years_with_company,age,num_total_claims,risk_level,annual_income_encoded,hh_size_est_encoded,consent_level,Average_Bought_Gap,Exceed Average Gap,Exceed Average Gap with Purchase,Months_Last_Bought_Category,Claimed_Large_Amount,Lapsed_Large_Amount,Average_Lapse_Gap,f_hold,flg_hlthclaim_ever,total_ape_purchased,total_sumins_purchased,total_prempaid_purchased
19550,Chinese,Singapore,P,ACTIVE,Female,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,1,0,0.000000,29.000000,0.000000,3,0.000000,0,0,1,nan,nan,nan,nan,nan,nan,nan,nan,None,nan,nan,nan,nan,nan,nan,nan,None,nan,None,None,None,None,nan,6,49,0.000000,1,4.000000,1,0,1.250000,False,False,<3 months,False,False,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4600,Chinese,Singapore,P,ACTIVE,Male,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,0,0,0.000000,140.000000,0.000000,1,0.000000,0,0,1,nan,nan,nan,nan,nan,nan,nan,nan,None,nan,nan,nan,nan,nan,nan,nan,None,nan,None,None,None,None,nan,16,44,0.000000,1,3.000000,3,0,47.500000,False,False,3-5 years,False,True,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
13337,Others,Singapore,P,ACTIVE,Male,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,1,0,0.000000,0.000000,0.000000,1,0.000000,0,0,1,nan,nan,nan,nan,nan,nan,nan,nan,None,nan,nan,nan,nan,nan,nan,nan,None,nan,None,None,None,None,nan,4,48,0.000000,1,6.000000,3,2,0.000000,True,False,3-5 years,False,False,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
15074,Chinese,Singapore,P,ACTIVE,Female,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,1,0,0.000000,0.000000,0.000000,1,0.000000,0,0,1,nan,nan,nan,nan,nan,nan,nan,nan,None,nan,nan,nan,nan,nan,nan,nan,None,nan,None,None,None,None,nan,2,47,0.000000,1,5.000000,4,0,0.000000,True,False,1-2 years,False,False,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
19724,Chinese,Singapore,P,ACTIVE,Female,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,1,0,0.000000,0.000000,0.000000,1,0.000000,0,0,1,nan,nan,nan,nan,nan,nan,nan,nan,None,nan,nan,nan,nan,nan,nan,nan,None,nan,None,None,None,None,nan,5,28,0.000000,1,2.000000,5,3,0.000000,True,False,5-10 years,True,False,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [371]:
print(df1.shape)
df1.head().style

(17992, 64)


,race_desc,ctrycode_desc,clttype,stat_flag,cltsex_fix,flg_is_revised_term,flg_is_rental_flat,flg_is_proposal,flg_with_preauthorisation,flg_is_returned_mail,is_housewife_retiree,is_sg_pr,is_class_1_2,is_dependent_in_at_least_1_policy,f_ever_declined_la,recency_lapse,recency_cancel,tot_inforce_pols,tot_cancel_pols,f_elx,f_mindef_mha,f_retail,flg_affconnect_show_interest_ever,flg_affconnect_ready_to_buy_ever,flg_affconnect_lapse_ever,affcon_visit_days,n_months_since_visit_affcon,clmcon_visit_days,recency_clmcon,recency_clmcon_regis,hlthclaim_amt,recency_hlthclaim,hlthclaim_cnt_success,recency_hlthclaim_success,hlthclaim_cnt_unsuccess,recency_hlthclaim_unsuccess,recency_hlthclaim_839f8a,recency_hlthclaim_14cb37,giclaim_amt,recency_giclaim,giclaim_cnt_success,recency_giclaim_success,giclaim_cnt_unsuccess,recency_giclaim_unsuccess,f_purchase_lh,years_with_company,age,num_total_claims,risk_level,annual_income_encoded,hh_size_est_encoded,consent_level,Average_Bought_Gap,Exceed Average Gap,Exceed Average Gap with Purchase,Months_Last_Bought_Category,Claimed_Large_Amount,Lapsed_Large_Amount,Average_Lapse_Gap,f_hold,flg_hlthclaim_ever,total_ape_purchased,total_sumins_purchased,total_prempaid_purchased
19550,Chinese,Singapore,P,ACTIVE,Female,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,1,0,0.000000,29.000000,0.000000,3,0.000000,0,0,1,nan,nan,nan,nan,nan,nan,nan,nan,None,nan,nan,nan,nan,nan,nan,nan,None,nan,None,None,None,None,nan,6,49,0.000000,1,4.000000,1,0,1.250000,False,False,<3 months,False,False,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4600,Chinese,Singapore,P,ACTIVE,Male,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,0,0,0.000000,140.000000,0.000000,1,0.000000,0,0,1,nan,nan,nan,nan,nan,nan,nan,nan,None,nan,nan,nan,nan,nan,nan,nan,None,nan,None,None,None,None,nan,16,44,0.000000,1,3.000000,3,0,47.500000,False,False,3-5 years,False,True,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
13337,Others,Singapore,P,ACTIVE,Male,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,1,0,0.000000,0.000000,0.000000,1,0.000000,0,0,1,nan,nan,nan,nan,nan,nan,nan,nan,None,nan,nan,nan,nan,nan,nan,nan,None,nan,None,None,None,None,nan,4,48,0.000000,1,6.000000,3,2,0.000000,True,False,3-5 years,False,False,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
15074,Chinese,Singapore,P,ACTIVE,Female,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,1,0,0.000000,0.000000,0.000000,1,0.000000,0,0,1,nan,nan,nan,nan,nan,nan,nan,nan,None,nan,nan,nan,nan,nan,nan,nan,None,nan,None,None,None,None,nan,2,47,0.000000,1,5.000000,4,0,0.000000,True,False,1-2 years,False,False,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
19724,Chinese,Singapore,P,ACTIVE,Female,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,1,0,0.000000,0.000000,0.000000,1,0.000000,0,0,1,nan,nan,nan,nan,nan,nan,nan,nan,None,nan,nan,nan,nan,nan,nan,nan,None,nan,None,None,None,None,nan,5,28,0.000000,1,2.000000,5,3,0.000000,True,False,5-10 years,True,False,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [372]:
# premium prepaid 
def sum_and_drop_columns(df, prefix, new_column_name):
    # Filter columns based on the specified prefix
    filtered_columns = df.filter(regex=f'^{prefix}', axis=1)
    
    # Get the column names
    sumins_columns = filtered_columns.columns
    
    # Calculate the sum along the horizontal axis and create a new column
    df[new_column_name] = filtered_columns.sum(axis=1)
    
    # Drop the original columns
    df.drop(columns=sumins_columns, inplace=True)

# ape 
sum_and_drop_columns(df1, 'ape', 'total_ape_purchased')
# sum_and_drop_columns(df1, 'ape_gi', 'total_ape_gi_purchased')
# sum_and_drop_columns(df1, 'ape_grp', 'total_ape_grp_purchased')
# sum_and_drop_columns(df1, 'ape_inv', 'total_ape_inv_purchased')
# sum_and_drop_columns(df1, 'ape_lh', 'total_ape_lh_purchased')
# sum_and_drop_columns(df1, 'ape_ltc', 'total_ape_ltc_purchased')
# sum_and_drop_columns(df1, 'ape_', 'total_ape_others_purchased')

# sumins 
sum_and_drop_columns(df1, 'sumins', 'total_sumins_purchased')
# sum_and_drop_columns(df1, 'sumins_gi', 'total_sumins_gi_purchased')
# sum_and_drop_columns(df1, 'sumins_grp', 'total_sumins_grp_purchased')
# sum_and_drop_columns(df1, 'sumins_inv', 'total_sumins_inv_purchased')
# sum_and_drop_columns(df1, 'sumins_lh', 'total_sumins_lh_purchased')
# sum_and_drop_columns(df1, 'sumins_ltc', 'total_sumins_ltc_purchased')
# sum_and_drop_columns(df1, 'sumins_', 'total_sumins_others_purchased')

# prempad 
sum_and_drop_columns(df1, 'prempaid', 'total_prempaid_purchased')
# sum_and_drop_columns(df1, 'prempaid_gi', 'total_prempaid_gi_purchased')
# sum_and_drop_columns(df1, 'prempaid_grp', 'total_prempaid_grp_purchased')
# sum_and_drop_columns(df1, 'prempaid_inv', 'total_prempaid_inv_purchased')
# sum_and_drop_columns(df1, 'prempaid_lh', 'total_prempaid_lh_purchased')
# sum_and_drop_columns(df1, 'prempaid_ltc', 'total_prempaid_ltc_purchased')
# sum_and_drop_columns(df1, 'prempaid_', 'total_prempaid_others_purchased')

In [373]:
print(df1.shape)
df1.head().style

(17992, 64)


,race_desc,ctrycode_desc,clttype,stat_flag,cltsex_fix,flg_is_revised_term,flg_is_rental_flat,flg_is_proposal,flg_with_preauthorisation,flg_is_returned_mail,is_housewife_retiree,is_sg_pr,is_class_1_2,is_dependent_in_at_least_1_policy,f_ever_declined_la,recency_lapse,recency_cancel,tot_inforce_pols,tot_cancel_pols,f_elx,f_mindef_mha,f_retail,flg_affconnect_show_interest_ever,flg_affconnect_ready_to_buy_ever,flg_affconnect_lapse_ever,affcon_visit_days,n_months_since_visit_affcon,clmcon_visit_days,recency_clmcon,recency_clmcon_regis,hlthclaim_amt,recency_hlthclaim,hlthclaim_cnt_success,recency_hlthclaim_success,hlthclaim_cnt_unsuccess,recency_hlthclaim_unsuccess,recency_hlthclaim_839f8a,recency_hlthclaim_14cb37,giclaim_amt,recency_giclaim,giclaim_cnt_success,recency_giclaim_success,giclaim_cnt_unsuccess,recency_giclaim_unsuccess,f_purchase_lh,years_with_company,age,num_total_claims,risk_level,annual_income_encoded,hh_size_est_encoded,consent_level,Average_Bought_Gap,Exceed Average Gap,Exceed Average Gap with Purchase,Months_Last_Bought_Category,Claimed_Large_Amount,Lapsed_Large_Amount,Average_Lapse_Gap,f_hold,flg_hlthclaim_ever,total_ape_purchased,total_sumins_purchased,total_prempaid_purchased
19550,Chinese,Singapore,P,ACTIVE,Female,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,1,0,0.000000,29.000000,0.000000,3,0.000000,0,0,1,nan,nan,nan,nan,nan,nan,nan,nan,None,nan,nan,nan,nan,nan,nan,nan,None,nan,None,None,None,None,nan,6,49,0.000000,1,4.000000,1,0,1.250000,False,False,<3 months,False,False,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4600,Chinese,Singapore,P,ACTIVE,Male,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,0,0,0.000000,140.000000,0.000000,1,0.000000,0,0,1,nan,nan,nan,nan,nan,nan,nan,nan,None,nan,nan,nan,nan,nan,nan,nan,None,nan,None,None,None,None,nan,16,44,0.000000,1,3.000000,3,0,47.500000,False,False,3-5 years,False,True,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
13337,Others,Singapore,P,ACTIVE,Male,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,1,0,0.000000,0.000000,0.000000,1,0.000000,0,0,1,nan,nan,nan,nan,nan,nan,nan,nan,None,nan,nan,nan,nan,nan,nan,nan,None,nan,None,None,None,None,nan,4,48,0.000000,1,6.000000,3,2,0.000000,True,False,3-5 years,False,False,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
15074,Chinese,Singapore,P,ACTIVE,Female,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,1,0,0.000000,0.000000,0.000000,1,0.000000,0,0,1,nan,nan,nan,nan,nan,nan,nan,nan,None,nan,nan,nan,nan,nan,nan,nan,None,nan,None,None,None,None,nan,2,47,0.000000,1,5.000000,4,0,0.000000,True,False,1-2 years,False,False,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
19724,Chinese,Singapore,P,ACTIVE,Female,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,1,0,0.000000,0.000000,0.000000,1,0.000000,0,0,1,nan,nan,nan,nan,nan,nan,nan,nan,None,nan,nan,nan,nan,nan,nan,nan,None,nan,None,None,None,None,nan,5,28,0.000000,1,2.000000,5,3,0.000000,True,False,5-10 years,True,False,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [374]:
test = df1.filter(regex='f_hold_.*')
print(test.shape)
test.head().style

(17992, 0)


19550
4600
13337
15074
19724
